In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [2]:
# Load Images
left_Computer = cv2.imread('computers_left.png', 0)
Right_Computer = cv2.imread('computers_right.png', 0)
left_DaftPunk = cv2.imread('daft_punk_left.jpg', 0)
Right_DaftPunk = cv2.imread('daft_punk_right.jpg', 0)


In [3]:
# 1.1 Feature Matching Helped make this with https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_epipolar_geometry/py_epipolar_geometry.html?fbclid=IwAR0fpsPcTQLmYJ6RdGzmuCp41ft09C2EMNbLnsxwSBkrzptb7hofJROIgGU
def ImageMatching(img1, img2):
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # FLANN parameters
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)

    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    pts1 = []
    pts2 = []
    good = []

    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance<0.8*n.distance:
            good.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)
    
    return pts1, pts2


In [4]:
# 1.2 Epipolar Lines Calculation Helped Make this with https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_epipolar_geometry/py_epipolar_geometry.html?fbclid=IwAR0fpsPcTQLmYJ6RdGzmuCp41ft09C2EMNbLnsxwSBkrzptb7hofJROIgGU

def drawlines(img1, img2, lines, pts1, pts2):
    rows, cols = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

    for row, pt1, pt2 in zip(lines, pts1, pts2):
        colour = tuple(np.random.randint(0,255,3).tolist())
        x1,y1 = map(int, [0, -row[2]/row[1]])
        x2,y2 = map(int, [cols, -((row[2]+row[0]*cols)/row[1])])
        #x2,y2 = abs(x2),abs(y2)
        
        #print(f"{x1},{y1},{x2},{y2}")

        img1 = cv2.line(img1, (x1,y1), (x2,y2), colour, 1)
        img1 = cv2.circle(img1, tuple(pt1), 5, colour, -1)
        img2 = cv2.circle(img2, tuple(pt2), 5, colour, -1)

    return img1, img2

# def drawlines(img1,img2,lines,pts1,pts2):
#     ''' img1 - image on which we draw the epilines for the points in img2
#         lines - corresponding epilines '''
#     r,c = img1.shape
#     img1 = cv.cvtColor(img1,cv.COLOR_GRAY2BGR)
#     img2 = cv.cvtColor(img2,cv.COLOR_GRAY2BGR)
#     for r,pt1,pt2 in zip(lines,pts1,pts2):
#         color = tuple(np.random.randint(0,255,3).tolist())
#         x0,y0 = map(int, [0, -r[2]/r[1] ])
#         x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
#         img1 = cv.line(img1, (x0,y0), (x1,y1), color,1)
#         img1 = cv.circle(img1,tuple(pt1),5,color,-1)
#         img2 = cv.circle(img2,tuple(pt2),5,color,-1)
#     return img1,img2
    
def lines_Epiporal(pts1, pts2):
    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)

    f, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_LMEDS)

    pts1 = pts1[mask.ravel()==1]
    pts2 = pts2[mask.ravel()==1]

    
    lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2, f).reshape(-1,3)
    lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1, f).reshape(-1,3)

    return pts1, pts2, lines1, lines2

In [5]:
#Displaying the Computer Images and there Connecting Points



left_pts_computer, right_pts_computer = ImageMatching(left_Computer, Right_Computer)
left_pts_computer2, right_pts_computer2, lines_l_comp, lines_r_comp = lines_Epiporal(left_pts_computer, right_pts_computer)


img3, img4 = drawlines(left_Computer, Right_Computer, lines_l_comp, left_pts_computer2, right_pts_computer2)
img5, img6 = drawlines(Right_Computer, left_Computer, lines_r_comp, right_pts_computer2, left_pts_computer2)


final_images = np.concatenate((img3, img5), axis=1)
cv2.imshow("Computers", final_images)
cv2.waitKey(0)



-1

In [6]:
#Displaying the Daftpunk Images and there Connecting Points



left_pts_punk, right_pts_punk = ImageMatching(left_DaftPunk, Right_DaftPunk)
left_pts_punk2, right_pts_punk2, lines_l_comp, lines_r_comp = lines_Epiporal(left_pts_punk, right_pts_punk)


img7, img8 = drawlines(left_DaftPunk, Right_DaftPunk, lines_l_comp, left_pts_punk2, right_pts_punk2)
img9,img10 = drawlines(Right_DaftPunk, left_DaftPunk, lines_r_comp, right_pts_punk2, left_pts_punk2)


final_images = np.concatenate((img7, img9), axis=1)
cv2.imshow("Daft Punk", final_images)
cv2.waitKey(0)

-1

In [117]:
#1.3
def pointLineSelection():
    pass
    #nah g, this a little to much work